# LLM Classification on EcoVerse Tasks Metrics and Utils


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [81]:
model_name='gemma-2b-it'
#model_name='Llama-2-13b-chat-hf'
#model_name='gpt-3.5-turbo-0125'

In [ ]:
for iteration in range(1, 101):
    # Define the paths
    input_file_path = f'/content/drive/MyDrive/EcoVerse_Results/{model_name}/stance/results_{iteration}.txt'
    modified_file_path = f'/content/drive/MyDrive/EcoVerse_Results/{model_name}/stance/formatted_results/results_{iteration}.txt'  # Change this if you want to overwrite the original file

    # Read the file, replace </s> with </s>\n, and save the modified content
    with open(input_file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    # Remove all newline characters
    #content = content.replace('*', ' ')
    content = content.replace('\n', '')

    # Add a newline character after each </s>
    content = content.replace('ID:', '\nID:')
    # Remove the first newline character
    content = content[1:]

    with open(modified_file_path, 'w', encoding='utf-8') as file:
        file.write(content)

print("Modification completed.")


In [ ]:
def normalize_probability(prob_str):
    # Check if the probability is expressed as a percentage
    if prob_str == '0.3.':
      prob_str = '0.3'
    if prob_str == '0.0.':
      prob_str = '0.0'
    if '%' in prob_str:
        # Remove the '%' sign and divide by 100
        return float(prob_str.replace('%', '')) / 100
    else:
        # Convert the probability to a float
        prob_float = float(prob_str)
        # If the probability is an integer from 1 to 100, divide by 100
        if prob_float > 1:
            return prob_float / 100
        # Otherwise, return the probability as is
        return prob_float

In [ ]:
import re
import json

for iteration in range(1, 101):
    # Define the input and output file paths
    input_file_path = f'/content/drive/MyDrive/EcoVerse_Results/{model_name}/formatted_results/results_{iteration}.txt'
    output_file_path = f'/content/drive/MyDrive/EcoVerse_Results/{model_name}/JSON/output_{iteration}.json'

    # Initialize a list to hold the extracted data
    data_list = []

    # Open and read the input file
    with open(input_file_path, 'r', encoding='utf-8') as file:
        for ID, line in zip(ids, file):
            # Extract the required fields using regular expressions
            id_match = re.search(r'ID: (\d+)', line)
            label_match = re.search(r'LABEL: eco-related: (YES|NO)', line)
            prob_no_match = re.search(r'NO_PROB=([\d\.]+%?|\d+%)', line)
            prob_yes_match = re.search(r'YES_PROB=([\d\.]+%?|\d+%)', line)
            exp_match = re.search(r'EXP: (.+)$', line)
            if prob_no_match:
              p_no = normalize_probability(prob_no_match.group(1))
            if prob_yes_match:
              p_yes = normalize_probability(prob_yes_match.group(1))

            # Check for matches and assign "NA" if no match is found
            id_val = ID
            label_val = 'eco-related' if label_match and label_match.group(1) == 'YES' else 'Not eco-related'
            prob_no_val = str(p_no) if prob_no_match else (str(1.0 - p_yes) if prob_yes_match else "0.0")
            prob_yes_val = str(p_yes) if prob_yes_match else (str(1.0 - p_no) if prob_no_match else "0.0")
            exp_val = exp_match.group(1) if exp_match else "NA"

            # Construct the dictionary for this line
            data = {
                'ID': id_val,
                'isgreen': label_val,
                'prob_yes': prob_yes_val,
                'prob_no': prob_no_val,
                'explanation': exp_val
            }
            data_list.append(data)

    # Write the extracted data to a JSON file
    with open(output_file_path, 'w') as json_file:
        json.dump(data_list, json_file, indent=4)


In [ ]:
import re
import json

for iteration in range(1, 101):
    # Define the input and output file paths
    input_file_path = f'/content/drive/MyDrive/EcoVerse_Results/{model_name}/eia/formatted_results/results_{iteration}.txt'
    output_file_path = f'/content/drive/MyDrive/EcoVerse_Results/{model_name}/eia/JSON/output_{iteration}.json'

    # Initialize a list to hold the extracted data
    data_list = []

    # Open and read the input file
    with open(input_file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # Extract the required fields using regular expressions
            id_match = re.search(r'ID: (\d+)', line)
            label_match = re.search(r'LABEL: (NEG|POS|NEU);', line)
            prob_pos_match = re.search(r'POS_PROB=([\d\.]+%?|\d+%)', line)
            prob_pos_match = re.search(r'POS_PROB=([\d\.]+%?|\d+%)', line)
            prob_neu_match = re.search(r'NEU_PROB=([\d\.]+%?|\d+%)', line)
            prob_neg_match = re.search(r'NEG_PROB=([\d\.]+%?|\d+%)', line)

            exp_match = re.search(r'EXP: (.+)$', line)
            if prob_pos_match:
              p_pos = normalize_probability(prob_pos_match.group(1))
            if prob_neu_match:
              p_neu = normalize_probability(prob_neu_match.group(1))
            if prob_neg_match:
              p_neg = normalize_probability(prob_neg_match.group(1))


            # Check for matches and assign "NA" if no match is found
            id_val = id_match.group(1)
            if label_match:
              if label_match.group(1) == 'POS':
                label_val = 'Positive'
              elif label_match.group(1) == 'NEU':
                label_val = 'Neutral'
              else:
                label_val = 'Negative'
            prob_pos_val = str(p_pos) if prob_pos_match else "NA"
            prob_neu_val = str(p_neu) if prob_neu_match else "NA"
            prob_neg_val = str(p_neg) if prob_neg_match else "NA"

            exp_val = exp_match.group(1) if exp_match else "NA"

            # Construct the dictionary for this line
            data = {
                'ID': id_val,
                'sentiment': label_val,
                'prob_pos': prob_pos_val,
                'prob_neu': prob_neu_val,
                'prob_neg': prob_neg_val,
                'explanation': exp_val
            }
            data_list.append(data)

    # Write the extracted data to a JSON file
    with open(output_file_path, 'w') as json_file:
        json.dump(data_list, json_file, indent=4)


In [ ]:
import re
import json

for iteration in range(1, 101):
    # Define the input and output file paths
    input_file_path = f'/content/drive/MyDrive/EcoVerse_Results/{model_name}/stance/formatted_results/results_{iteration}.txt'
    output_file_path = f'/content/drive/MyDrive/EcoVerse_Results/{model_name}/stance/JSON/output_{iteration}.json'

    # Initialize a list to hold the extracted data
    data_list = []

    # Open and read the input file
    with open(input_file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # Extract the required fields using regular expressions
            id_match = re.search(r'ID: (\d+)', line)
            label_match = re.search(r'LABEL: (SKE|SUP|NEU);', line)
            prob_pos_match = re.search(r'SUP_PROB=([\d\.]+%?|\d+%)', line)
            prob_neu_match = re.search(r'NEU_PROB=([\d\.]+%?|\d+%)', line)
            prob_neg_match = re.search(r'SKE_PROB=([\d\.]+%?|\d+%)', line)

            exp_match = re.search(r'EXP: (.+)$', line)
            if prob_pos_match:
              p_pos = normalize_probability(prob_pos_match.group(1))
            if prob_neu_match:
              p_neu = normalize_probability(prob_neu_match.group(1))
            if prob_neg_match:
              p_neg = normalize_probability(prob_neg_match.group(1))


            # Check for matches and assign "NA" if no match is found
            id_val = id_match.group(1)
            if label_match:
              if label_match.group(1) == 'SUP':
                label_val = 'Supportive'
              elif label_match.group(1) == 'NEU':
                label_val = 'Neutral'
              else:
                label_val = 'Skeptical/Opposing'
            prob_pos_val = str(p_pos) if prob_pos_match else "NA"
            prob_neu_val = str(p_neu) if prob_neu_match else "NA"
            prob_neg_val = str(p_neg) if prob_neg_match else "NA"

            exp_val = exp_match.group(1) if exp_match else "NA"

            # Construct the dictionary for this line
            data = {
                'ID': id_val,
                'stance': label_val,
                'prob_sup': prob_pos_val,
                'prob_neu': prob_neu_val,
                'prob_ske': prob_neg_val,
                'explanation': exp_val
            }
            data_list.append(data)

    # Write the extracted data to a JSON file
    with open(output_file_path, 'w') as json_file:
        json.dump(data_list, json_file, indent=4)


In [ ]:
output_file_path = f'/content/drive/MyDrive/EcoVerse_Results/{model_name}/stance/JSON/output_100.json'

# Load the JSON data from the file
with open(output_file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# Count the number of elements in the JSON data
num_elements = len(data)
print(num_elements)

In [ ]:
import json

# Path to your JSON file
json_file_path = f'/content/drive/MyDrive/EcoVerse_Results/{model_name}/JSON/output_1.json'

# Initialize a list to hold the IDs
eco_related_ids = []

# Load the JSON data from the file
with open(json_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

    # Assuming the data is a list of dictionaries, each with an 'ID' field
    for item in data:
        # Append the ID to the list, use a default value if ID is missing
        is_eco_rel = item.get('isgreen') == 'eco-related'
        if is_eco_rel:
          eco_related_ids.append(int(item.get('ID')))

print((eco_related_ids))

# Now ids contains all the ID values from the JSON file

# Metrics

## Eco-relevance

In [80]:
from sklearn.metrics import classification_report
import numpy as np

#eco_rel_labels = df['isgreen'].tolist()
#eia_labels = df['sentiment'].tolist()
#stance_labels = df['stance'].tolist()

eco_rel_labels = [e.lower() for e in eco_rel_labels]

mod = 'gemma-2b-it'
target_names=['not eco-related', 'eco-related']
accuracy_list, precision_list, recall_list, f1_list = [], [], [], []


for iteration in range(1, 11):
  json_file_path = f'/content/drive/MyDrive/EcoVerse_Results/{mod}/JSON/output_{iteration}.json'
  eco_rel_pred = []
  with open(json_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

    # Assuming the data is a list of dictionaries, each with an 'ID' field
    for item in data:
        # Append the ID to the list, use a default value if ID is missing
          eco_rel_pred.append(item.get('isgreen').lower())
    #print(classification_report(eco_rel_labels, eco_rel_pred, target_names=target_names))
    report = classification_report(eco_rel_labels, eco_rel_pred, target_names=target_names, output_dict=True)
    accuracy_list.append(report['accuracy'])
    precision_list.append(report['not eco-related']['precision'])
    recall_list.append(report['eco-related']['recall'])
    f1_list.append(report['eco-related']['f1-score'])

mean_accuracy = np.mean(accuracy_list)
mean_precision = np.mean(precision_list)
mean_recall = np.mean(recall_list)
mean_f1 = np.mean(f1_list)

print(f"Mean Accuracy for model {mod} on Eco-Relevance task: {mean_accuracy}")
print(f"Mean Precision for model {mod} on Eco-Relevance task: {mean_precision}")
print(f"Mean Recall for model {mod} on Eco-Relevance task: {mean_recall}")
print(f"Mean F1-Score for model {mod} on Eco-Relevance task: {mean_f1}")
#print(classification_report(y_true, y_pred, target_names=target_names))


Mean Accuracy for model gemma-2b-it on Eco-Relevance task: 0.49537953795379536
Mean Precision for model gemma-2b-it on Eco-Relevance task: 0.550619777835605
Mean Recall for model gemma-2b-it on Eco-Relevance task: 0.39166666666666666
Mean F1-Score for model gemma-2b-it on Eco-Relevance task: 0.40335404683160797


## EIA

In [79]:
from sklearn.metrics import classification_report
import numpy as np

#eco_rel_labels = df['isgreen'].tolist()
#eia_labels = df['sentiment'].tolist()
#stance_labels = df['stance'].tolist()


mod = model_name


target_names=['Positive', 'Neutral', 'Negative']
accuracy = []
precision_pos_list, precision_neu_list, precision_neg_list = [], [], []
recall_pos_list, recall_neu_list, recall_neg_list = [], [], []
f1_pos_list, f1_neu_list, f1_neg_list = [], [], []
precision_macro_list, recall_macro_list, f1_macro_list = [], [], []

for iteration in range(1, 11):
  json_file_path = f'/content/drive/MyDrive/EcoVerse_Results/{mod}/eia/JSON/output_{iteration}.json'
  eia_pred = []
  ids_pred = []
  with open(json_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

    # Assuming the data is a list of dictionaries, each with an 'ID' field
    for item in data:
      ids_pred.append(int(item.get('ID')))
      eia_pred.append(item.get('sentiment').lower())

    data_real = df[(df['id'].isin(ids_pred)) & (df['sentiment'].notna())]
    rem_ids = data_real['id'].tolist()
    eia_pred = []
    for item in data:
      if int(item.get('ID')) in rem_ids:
        eia_pred.append(item.get('sentiment'))

    eia_labels = data_real['sentiment'].dropna().tolist()

    #print(classification_report(eia_labels, eia_pred, target_names=target_names))
    report = classification_report(eia_labels, eia_pred, target_names=target_names, output_dict=True)
    accuracy.append(report['accuracy'])
    precision_pos_list.append(report['Positive']['precision'])
    recall_pos_list.append(report['Positive']['recall'])
    f1_pos_list.append(report['Positive']['f1-score'])

    precision_neu_list.append(report['Neutral']['precision'])
    recall_neu_list.append(report['Neutral']['recall'])
    f1_neu_list.append(report['Neutral']['f1-score'])

    precision_neg_list.append(report['Negative']['precision'])
    recall_neg_list.append(report['Negative']['recall'])
    f1_neg_list.append(report['Negative']['f1-score'])

    precision_macro_list.append(report['macro avg']['precision'])
    recall_macro_list.append(report['macro avg']['recall'])
    f1_macro_list.append(report['macro avg']['f1-score'])



mean_accuracy = np.mean(accuracy)

mean_precision_pos = np.mean(precision_pos_list)
mean_recall_pos = np.mean(recall_pos_list)
mean_f1_pos = np.mean(f1_pos_list)

mean_precision_neu = np.mean(precision_neu_list)
mean_recall_neu = np.mean(recall_neu_list)
mean_f1_neu = np.mean(f1_neu_list)

mean_precision_neg = np.mean(precision_neg_list)
mean_recall_neg = np.mean(recall_neg_list)
mean_f1_neg = np.mean(f1_neg_list)

    # Now, calculate the mean for each macro avg metric
mean_precision_macro = np.mean(precision_macro_list)
mean_recall_macro = np.mean(recall_macro_list)
mean_f1_macro = np.mean(f1_macro_list)


# Printing mean metrics for each label
print(f"Mean Accuracy for model {mod} on EIA task: {mean_accuracy}\n")

print(f"Mean Macro Avg Precision for model {mod} on EIA task: {mean_precision_macro}")
print(f"Mean Macro Avg Recall for model {mod} on EIA task: {mean_recall_macro}")
print(f"Mean Macro Avg F1-Score for model {mod} on EIA task: {mean_f1_macro}")

print(f"Mean Precision for model {mod} on EIA task (Positive): {mean_precision_pos}")
print(f"Mean Recall for model {mod} on EIA task (Positive): {mean_recall_pos}")
print(f"Mean F1-Score for model {mod} on EIA task (Positive): {mean_f1_pos}\n")

print(f"Mean Precision for model {mod} on EIA task (Neutral): {mean_precision_neu}")
print(f"Mean Recall for model {mod} on EIA task (Neutral): {mean_recall_neu}")
print(f"Mean F1-Score for model {mod} on EIA task (Neutral): {mean_f1_neu}\n")

print(f"Mean Precision for model {mod} on EIA task (Negative): {mean_precision_neg}")
print(f"Mean Recall for model {mod} on EIA task (Negative): {mean_recall_neg}")
print(f"Mean F1-Score for model {mod} on EIA task (Negative): {mean_f1_neg}")

Mean Accuracy for model gpt-3.5-turbo-0125 on EIA task: 0.8406779661016948

Mean Macro Avg Precision for model gpt-3.5-turbo-0125 on EIA task: 0.6855595068752963
Mean Macro Avg Recall for model gpt-3.5-turbo-0125 on EIA task: 0.6668987168987168
Mean Macro Avg F1-Score for model gpt-3.5-turbo-0125 on EIA task: 0.667537165590874
Mean Precision for model gpt-3.5-turbo-0125 on EIA task (Positive): 0.8337837837837837
Mean Recall for model gpt-3.5-turbo-0125 on EIA task (Positive): 0.9121212121212119
Mean F1-Score for model gpt-3.5-turbo-0125 on EIA task (Positive): 0.8711801242236025

Mean Precision for model gpt-3.5-turbo-0125 on EIA task (Neutral): 0.34
Mean Recall for model gpt-3.5-turbo-0125 on EIA task (Neutral): 0.18181818181818182
Mean F1-Score for model gpt-3.5-turbo-0125 on EIA task (Neutral): 0.23676470588235293

Mean Precision for model gpt-3.5-turbo-0125 on EIA task (Negative): 0.8828947368421053
Mean Recall for model gpt-3.5-turbo-0125 on EIA task (Negative): 0.9067567567567567

## Stance

In [78]:
from sklearn.metrics import classification_report
import numpy as np

#eco_rel_labels = df['isgreen'].tolist()
#eia_labels = df['sentiment'].tolist()
#stance_labels = df['stance'].tolist()


#mod = 'Llama-2-13b-chat-hf'
#mod = 'gemma-2b-it'
mod = 'gpt-3.5-turbo-0125'

target_names=['Supportive', 'Neutral', 'Skeptical/Opposing']
accuracy = []
precision_pos_list, precision_neu_list, precision_neg_list = [], [], []
recall_pos_list, recall_neu_list, recall_neg_list = [], [], []
f1_pos_list, f1_neu_list, f1_neg_list = [], [], []
precision_macro_list, recall_macro_list, f1_macro_list = [], [], []

for iteration in range(1, 11):
  json_file_path = f'/content/drive/MyDrive/EcoVerse_Results/{mod}/stance/JSON/output_{iteration}.json'
  stance_pred = []
  ids_pred = []
  with open(json_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

    # Assuming the data is a list of dictionaries, each with an 'ID' field
    for item in data:
      ids_pred.append(int(item.get('ID')))
      stance_pred.append(item.get('stance').lower())

    data_real = df[(df['id'].isin(ids_pred)) & (df['stance'].notna())]
    rem_ids = data_real['id'].tolist()
    stance_pred = []
    for item in data:
      if int(item.get('ID')) in rem_ids:
        stance_pred.append(item.get('stance'))

    stance_labels = data_real['stance'].dropna().tolist()

    #print(classification_report(eco_rel_labels, eco_rel_pred, target_names=target_names))
    report = classification_report(stance_labels, stance_pred, target_names=target_names, output_dict=True)
    accuracy.append(report['accuracy'])
    precision_pos_list.append(report['Supportive']['precision'])
    recall_pos_list.append(report['Supportive']['recall'])
    f1_pos_list.append(report['Supportive']['f1-score'])

    precision_neu_list.append(report['Neutral']['precision'])
    recall_neu_list.append(report['Neutral']['recall'])
    f1_neu_list.append(report['Neutral']['f1-score'])

    precision_neg_list.append(report['Skeptical/Opposing']['precision'])
    recall_neg_list.append(report['Skeptical/Opposing']['recall'])
    f1_neg_list.append(report['Skeptical/Opposing']['f1-score'])

    precision_macro_list.append(report['macro avg']['precision'])
    recall_macro_list.append(report['macro avg']['recall'])
    f1_macro_list.append(report['macro avg']['f1-score'])



mean_accuracy = np.mean(accuracy)

mean_precision_pos = np.mean(precision_pos_list)
mean_recall_pos = np.mean(recall_pos_list)
mean_f1_pos = np.mean(f1_pos_list)

mean_precision_neu = np.mean(precision_neu_list)
mean_recall_neu = np.mean(recall_neu_list)
mean_f1_neu = np.mean(f1_neu_list)

mean_precision_neg = np.mean(precision_neg_list)
mean_recall_neg = np.mean(recall_neg_list)
mean_f1_neg = np.mean(f1_neg_list)

    # Now, calculate the mean for each macro avg metric
mean_precision_macro = np.mean(precision_macro_list)
mean_recall_macro = np.mean(recall_macro_list)
mean_f1_macro = np.mean(f1_macro_list)


# Printing mean metrics for each label
print(f"Mean Accuracy for model {mod} on stance task: {mean_accuracy}\n")

print(f"Mean Macro Avg Precision for model {mod} on stance task: {mean_precision_macro}")
print(f"Mean Macro Avg Recall for model {mod} on stance task: {mean_recall_macro}")
print(f"Mean Macro Avg F1-Score for model {mod} on stance task: {mean_f1_macro}")

print(f"Mean Precision for model {mod} on EIA task (Positive): {mean_precision_pos}")
print(f"Mean Recall for model {mod} on EIA task (Positive): {mean_recall_pos}")
print(f"Mean F1-Score for model {mod} on EIA task (Positive): {mean_f1_pos}\n")

print(f"Mean Precision for model {mod} on EIA task (Neutral): {mean_precision_neu}")
print(f"Mean Recall for model {mod} on EIA task (Neutral): {mean_recall_neu}")
print(f"Mean F1-Score for model {mod} on EIA task (Neutral): {mean_f1_neu}\n")

print(f"Mean Precision for model {mod} on EIA task (Negative): {mean_precision_neg}")
print(f"Mean Recall for model {mod} on EIA task (Negative): {mean_recall_neg}")
print(f"Mean F1-Score for model {mod} on EIA task (Negative): {mean_f1_neg}")

Mean Accuracy for model gpt-3.5-turbo-0125 on stance task: 0.6587301587301588

Mean Macro Avg Precision for model gpt-3.5-turbo-0125 on stance task: 0.5684678115807869
Mean Macro Avg Recall for model gpt-3.5-turbo-0125 on stance task: 0.7065733955141018
Mean Macro Avg F1-Score for model gpt-3.5-turbo-0125 on stance task: 0.5628044053136032
Mean Precision for model gpt-3.5-turbo-0125 on EIA task (Positive): 0.5583388962084614
Mean Recall for model gpt-3.5-turbo-0125 on EIA task (Positive): 0.3526315789473684
Mean F1-Score for model gpt-3.5-turbo-0125 on EIA task (Positive): 0.43148249484488294

Mean Precision for model gpt-3.5-turbo-0125 on EIA task (Neutral): 0.2873995244270223
Mean Recall for model gpt-3.5-turbo-0125 on EIA task (Neutral): 1.0
Mean F1-Score for model gpt-3.5-turbo-0125 on EIA task (Neutral): 0.4462703171384546

Mean Precision for model gpt-3.5-turbo-0125 on EIA task (Negative): 0.8596650141068768
Mean Recall for model gpt-3.5-turbo-0125 on EIA task (Negative): 0.76708

# Calibration

## Cal Eco-Relevance

In [1]:
!pip install -U scikit-learn

In [3]:
!pip install scipy


In [82]:
from sklearn.metrics import precision_score
from scipy.stats import binned_statistic
import numpy as np
import json

# Initialize lists for probabilities and labels
eco_rel_probs = []
eco_rel_labels = []  # Assuming binary labels (0 for 'not eco-related', 1 for 'eco-related')

# Example loop to populate lists - replace with actual data loading as needed
for iteration in range(1, 11):
    json_file_path = f'/content/drive/MyDrive/EcoVerse_Results/{model_name}/JSON/output_{iteration}.json'
    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        for item in data:
            eco_rel_probs.append(float(item.get('prob_no')))
            eco_rel_labels.append(1 if item.get('isgreen').lower() == 'not eco-related' else 0)

# Convert lists to numpy arrays for efficient computation
eco_rel_probs = np.array(eco_rel_probs)
eco_rel_labels = np.array(eco_rel_labels)

# Define bins and bin labels for probability ranges
bins = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
bin_labels = ['0-0.2', '0.2-0.4', '0.4-0.6', '0.6-0.8', '0.8-1.0']

# Digitize the probabilities into bins
digitized_probs = np.digitize(eco_rel_probs, bins) - 1

# Calculate mean probability and precision for each bin
for i in range(len(bins)-1):
    bin_indices = (digitized_probs == i)
    if bin_indices.any():
        bin_mean_prob = np.mean(eco_rel_probs[bin_indices])
        bin_precision = precision_score(eco_rel_labels[bin_indices], eco_rel_probs[bin_indices] >= 0.5)
        print(f"Bin {bin_labels[i]} - Mean Probability: {bin_mean_prob}, Precision: {bin_precision}")

Bin 0-0.2 - Mean Probability: 0.03632450157397692, Precision: 0.0
Bin 0.2-0.4 - Mean Probability: 0.2517596566523605, Precision: 0.0
Bin 0.4-0.6 - Mean Probability: 0.502, Precision: 0.2
Bin 0.6-0.8 - Mean Probability: 0.7443655172413793, Precision: 0.8620689655172413
Bin 0.8-1.0 - Mean Probability: 0.9395314285714285, Precision: 0.32571428571428573


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Cal eia

In [ ]:
from sklearn.metrics import precision_score
from scipy.stats import binned_statistic
import numpy as np
import json

# Initialize lists for probabilities and labels
sentiment_probs = []
sentiment_labels = []  # Assuming binary labels (0 for 'not eco-related', 1 for 'eco-related')

# Example loop to populate lists - replace with actual data loading as needed
for iteration in range(1, 101):
    json_file_path = f'/content/drive/MyDrive/EcoVerse_Results/{model_name}/eia/JSON/output_{iteration}.json'
    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        for item in data:
            if item.get('prob_neg') is not None and item.get('prob_neg') != "NA":
              sentiment_probs.append(float(item.get('prob_neg')))
              sentiment_labels.append(1 if item.get('sentiment').lower() == 'neg' else 0)
# Convert lists to numpy arrays for efficient computation
sentiment_probs = np.array(sentiment_probs)
sentiment_labels = np.array(sentiment_labels)
# Define bins and bin labels for probability ranges
bins = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
bin_labels = ['0-0.2', '0.2-0.4', '0.4-0.6', '0.6-0.8', '0.8-1.0']

# Digitize the probabilities into bins
digitized_probs = np.digitize(sentiment_probs, bins) - 1

# Calculate mean probability and precision for each bin
for i in range(len(bins)-1):
    bin_indices = (digitized_probs == i)
    if bin_indices.any():
        bin_mean_prob = np.mean(sentiment_probs[bin_indices])
        bin_precision = precision_score(sentiment_labels[bin_indices], sentiment_probs[bin_indices] >= 0.5)
        print(f"Bin {bin_labels[i]} - Mean Probability: {bin_mean_prob}, Precision: {bin_precision}")

In [ ]:
import numpy as np
import json
from sklearn.metrics import precision_score
from sklearn.preprocessing import label_binarize

bins = np.linspace(0, 1, 6)
bin_labels = ['0-0.2', '0.2-0.4', '0.4-0.6', '0.6-0.8', '0.8-1.0']
sentiments = ['pos', 'neu', 'neg']

precision_bins = {sentiment: {label: [] for label in bin_labels} for sentiment in sentiments}
bin_mean_probs = {sentiment: {label: [] for label in bin_labels} for sentiment in sentiments}

for iteration in range(1, 11):
    json_file_path = f'/content/drive/MyDrive/EcoVerse_Results/{model_name}/eia/JSON/output_{iteration}.json'
    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Initialize or clear lists for predicted labels and probabilities for this iteration
    eia_pred, pos_probs, neu_probs, neg_probs = [], [], [], []
    for item in data:
        sentiment_pred = item.get('sentiment').lower()
        eia_pred.append(sentiment_pred)
        pos_probs.append(float(item.get('prob_pos')))
        neu_probs.append(float(item.get('prob_neu')))
        neg_probs.append(float(item.get('prob_neg')))

    # Assume eia_labels are loaded with the correct labels for this iteration
    eia_labels_lower = [label.lower() for label in eia_labels]  # Ensure lowercase for consistency
    for sentiment in sentiments:
        sentiment_probs = {
            'pos': pos_probs,
            'neu': neu_probs,
            'neg': neg_probs
        }[sentiment]

        bin_indices = np.digitize(sentiment_probs, bins) - 1  # Assign probabilities to bins

        for i, bin_range in enumerate(bin_labels):
            in_bin = bin_indices == i
            if np.any(in_bin):
                # Filter predictions and true labels for current bin
                bin_preds = [eia_pred[j] for j in range(len(eia_pred)) if in_bin[j]]
                bin_true = [eia_labels_lower[j] for j in range(len(eia_labels_lower)) if in_bin[j]]

                # Binarize labels for precision calculation
                bin_preds_binarized = label_binarize(bin_preds, classes=sentiments)[:, sentiments.index(sentiment)]
                bin_true_binarized = label_binarize(bin_true, classes=sentiments)[:, sentiments.index(sentiment)]

                precision = precision_score(bin_true_binarized, bin_preds_binarized, zero_division=0)
                precision_bins[sentiment][bin_range].append(precision)
                bin_mean_probs[sentiment][bin_range].append(np.mean(sentiment_probs))

# Calculate and print mean precision and probability for each bin and sentiment
for sentiment in sentiments:
    print(f"\nSentiment: {sentiment.capitalize()}")
    for bin_range in bin_labels:
        mean_precision = np.mean(precision_bins[sentiment][bin_range], default=0)
        mean_prob = np.mean(bin_mean_probs[sentiment][bin_range], default=0)
        print(f"Bin {bin_range}, Mean Probability: {mean_prob:.4f} - Mean Precision: {mean_precision:.4f}")

## cal stance

In [ ]:
from sklearn.metrics import precision_score
from scipy.stats import binned_statistic
import numpy as np
import json

# Initialize lists for probabilities and labels
sentiment_probs = []
sentiment_labels = []  # Assuming binary labels (0 for 'not eco-related', 1 for 'eco-related')

# Example loop to populate lists - replace with actual data loading as needed
for iteration in range(1, 11):
    json_file_path = f'/content/drive/MyDrive/EcoVerse_Results/{model_name}/stance/JSON/output_{iteration}.json'
    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        for item in data:
            if item.get('prob_neu') is not None and item.get('prob_neu') != "NA":
              sentiment_probs.append(float(item.get('prob_neu')))
              sentiment_labels.append(1 if item.get('stance').lower() == 'neu' else 0)
# Convert lists to numpy arrays for efficient computation
sentiment_probs = np.array(sentiment_probs)
sentiment_labels = np.array(sentiment_labels)
# Define bins and bin labels for probability ranges
bins = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
bin_labels = ['0-0.2', '0.2-0.4', '0.4-0.6', '0.6-0.8', '0.8-1.0']

# Digitize the probabilities into bins
digitized_probs = np.digitize(sentiment_probs, bins) - 1

# Calculate mean probability and precision for each bin
for i in range(len(bins)-1):
    bin_indices = (digitized_probs == i)
    if bin_indices.any():
        bin_mean_prob = np.mean(sentiment_probs[bin_indices])
        bin_precision = precision_score(sentiment_labels[bin_indices], sentiment_probs[bin_indices] >= 0.5)
        print(f"Bin {bin_labels[i]} - Mean Probability: {bin_mean_prob}, Precision: {bin_precision}")